In [13]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
import tensorflow_datasets as tfds

import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dropout
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
import tensorflow.keras.backend as K
from sklearn.utils.class_weight import compute_class_weight


import warnings
warnings.filterwarnings("ignore")

In [14]:
num_clients=5
client_names = ['{}_{}'.format('client', i+1) for i in range(num_clients)]
comms_round = 2

## Useful methods

In [15]:
def create_clients_equ(X, y):
    ''' return: a dictionary with keys clients' names and value as 
                data shards - tuple of data and label lists.  '''  

    train_data = X.copy(deep=True)
    train_data['has_diabetes'] = y
    
    pos_data = train_data[train_data['has_diabetes']==1]
    neg_data = train_data[train_data['has_diabetes']==0]

    pos_size = len(pos_data) // num_clients
    neg_size = len(neg_data) // num_clients

    pos_shards = [pos_data[i:i + pos_size] for i in range(0, pos_size * num_clients, pos_size)]
    neg_shards = [neg_data[i:i + neg_size] for i in range(0, neg_size * num_clients, neg_size)]

    return {client_names[i] : pos_shards[i].append(neg_shards[i]) for i in range(len(client_names))}

In [16]:
def create_clients(X, y):
    ''' return: a dictionary with keys clients' names and value as 
                data shards - tuple of data and label lists.  '''  
    train_data = X.copy(deep=True)
    train_data['has_diabetes'] = y

    size = len(train_data)//num_clients
    shards = [train_data[i:i + size] for i in range(0, size*num_clients, size)]

    return {client_names[i] : shards[i] for i in range(len(client_names))}

In [17]:
def batch_data(data_shard, bs=32):
    '''Takes in a clients data shard and batch size; return a Tensorflow dataset object off it'''
    
    #seperate shard into data and labels lists
    data, label = data_shard.drop('has_diabetes', axis=1), data_shard['has_diabetes'] 
    dataset = tf.data.Dataset.from_tensor_slices((data.values, label.values))
    return dataset.shuffle(len(label)).batch(bs)

In [18]:
def create_model():
    model = keras.Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(256,activation="relu"),
        Dropout(0.3),
        Dense(128, activation="relu"),
        Dropout(0.3),
        Dense(64, activation="relu"),
        Dropout(0.3),
        Dense(1, activation="sigmoid")
    ])
    return model

loss='binary_crossentropy'
metrics = ['accuracy']      

In [19]:
def weight_scalling_factor(clients_trn_data, client_name):
    client_names = list(clients_trn_data.keys())
    # bs = list(clients_trn_data[client_name])[0][0].shape[0]
    
    global_count = sum([len(clients_trn_data[client_name]) for client_name in client_names])
    local_count =  len(clients_trn_data[client_name])

    # global_count = sum([tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy() for client_name in client_names]) *bs
    # local_count = tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy()*bs
    return local_count/global_count

def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final



def sum_scaled_weights(scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
        
    return avg_grad


In [20]:
def test_model(X_test, Y_test,  model, comm_round):
    predicted = model.predict(X_test)    
    loss, acc = model.evaluate(X_test, Y_test, verbose=0)
    print('comm_round: {} | global_acc: {:.2%} | global_loss: {:.2f}'.format(comm_round, acc, loss))
    return acc, loss

## Prepare data

In [21]:
data = pd.read_csv('LLCP2022_filtered.csv')
data.drop_duplicates(inplace=True)
data = data.astype(np.float32)

data['has_diabetes'] = data['has_diabetes'].replace({2.0: 0.0})
data['has_diabetes'] = data['has_diabetes'].astype(int)

y = data['has_diabetes']
X = data.drop('has_diabetes', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Prepare clients

In [22]:
clients = create_clients_equ(X_train, y_train)
clients_batched = clients

In [23]:
K.clear_session()

## Prepare and fit models

In [24]:
smlp_global = create_model()
smlp_global.build(input_shape=(None, X_train.shape[1]))
smlp_global.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), metrics=metrics)

global_model = smlp_global
scaling_factor = 100/num_clients


for comm_round in range(comms_round):
    print(f"Round {comm_round+1}")        
    
    global_weights = global_model.get_weights()
    
    scaled_local_weight_list = list()
    
    #loop through each client and create new local model
    for client in client_names:  
        smlp_local = create_model()
        smlp_local.build(input_shape=(None, X_train.shape[1]))
        local_model = smlp_local
        local_model.compile(loss=loss,  
                      optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
                      metrics=metrics)
        
        
        local_model.set_weights(global_weights)

        # y_client = np.concatenate([labels.numpy() for _, labels in clients_batched[client]])
        x_client = clients_batched[client].drop('has_diabetes', axis=1)
        y_client = clients_batched[client]['has_diabetes']

        class_weights = compute_class_weight( class_weight='balanced', classes=np.unique(y_client), y=y_client)
        class_weights_dict = dict(enumerate(class_weights)) 
    
        local_model.fit(x=x_client.to_numpy(),y=y_client.to_numpy(), epochs=2, verbose=0,class_weight=class_weights_dict)
        
        local_loss, local_accuracy = local_model.evaluate(X_test, y_test, verbose=0)
        predicted = local_model.predict(X_test,verbose=0) 
  
        print(f"   {client}: acc = {local_accuracy:.2%}, loss = {local_loss:.2f}, pred_1 = {np.sum(predicted>=0.5)}/{np.count_nonzero(y_test==1)}, pred_0 = {np.sum(predicted<0.5)}/{np.count_nonzero(y_test==0)}  ")
    
        #scale the model weights and add to list
        # scaling_factor = weight_scalling_factor(clients_batched, client)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)


    K.clear_session()
    
    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights = sum_scaled_weights(scaled_local_weight_list)

    global_model.set_weights(average_weights)
    test_model(X_test, y_test, global_model,comm_round)
    

Round 1
   client_1: acc = 65.15%, loss = 1.06, pred_1 = 6257/9761, pred_0 = 34116/30612  
   client_2: acc = 29.64%, loss = 1.11, pred_1 = 34496/9761, pred_0 = 5877/30612  
   client_3: acc = 25.88%, loss = 1.19, pred_1 = 38429/9761, pred_0 = 1944/30612  
   client_4: acc = 75.82%, loss = 0.68, pred_1 = 1/9761, pred_0 = 40372/30612  
   client_5: acc = 73.86%, loss = 1.12, pred_1 = 1979/9761, pred_0 = 38394/30612  
1262/1262 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step
comm_round: 0 | global_acc: 44.66% | global_loss: 31688456.00
Round 2
   client_1: acc = 48.12%, loss = 30960306.00, pred_1 = 17589/9761, pred_0 = 22784/30612  
   client_2: acc = 48.88%, loss = 30615940.00, pred_1 = 17003/9761, pred_0 = 23370/30612  
   client_3: acc = 48.34%, loss = 30837090.00, pred_1 = 17419/9761, pred_0 = 22954/30612  
   client_4: acc = 52.73%, loss = 30016540.00, pred_1 = 14078/9761, pred_0 = 26295/30612  
   client_5: acc = 49.32%, loss = 30269818.00, pred_1 = 16654/9761, pred_0 = 23719/30612  
1262/1262 ━━